In [16]:
import sys
# sys.path.append('./keras-spp/')
# from spp.SpatialPyramidPooling import SpatialPyramidPooling

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import glob
import os
from scipy.misc import imread, imresize
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2gray
from keras.utils import np_utils
import pywt
import matplotlib.pyplot as plt
import cv2
from skimage.transform import resize


#### Load Data

In [37]:
master_folder = '../GrantData401Project4/PhotosDataset/'

imgs = []
labels = []
for photo in os.listdir(master_folder+'/Alex'):
    img = imread(master_folder+'/Alex/'+photo)
    imgs.append(resize(img,(200,200,4)))
    labels.append('Alex')
for photo in os.listdir(master_folder+'/Hunter'):
    img = imread(master_folder+'/Hunter/'+photo)
    imgs.append(resize(img,(200,200,4)))
    labels.append('Hunter')

c:\users\gjber\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
c:\users\gjber\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
c:\users\gjber\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
c:\users\gjber\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will

#### Massage Data

In [38]:
# Take out alpha component of image
imgs = [img[:,:,[0,1,2]] for img in imgs]

labels = np.array([[1,0] if label is 'Hunter' else [0,1] for label in labels])

x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=.5,
                                                    stratify=labels)

In [40]:
x_train = np.stack(x_train,axis=0)
x_test = np.stack(x_test,axis=0)

num_classes = 2
# y_train = np_utils.to_categorical(y_train, num_classes)
# y_test = np_utils.to_categorical(y_test, num_classes)

In [41]:
# # Use a generator to feed data because data images are of different dimensions so a numpy array cant
# # be constructed
# def generator(x, y):
#     while True:
#         for i,img in enumerate(x):
#             yield np.expand_dims(img,axis=0), np.expand_dims(y[i],axis=0)
            

            
# gen = generator(x_train, y_train)
# val_gen = generator(x_test,y_test)
# num_channels = 3
# num_classes = 2
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(x_train)
# gen = generator(x_train, y_train)
# val_gen = generator(x_test,y_test)

batch_size=4
epochs=10


### Model with no transfer learning

In [42]:
model = Sequential()

# Note that we leave the image size as None to allow multiple image sizes
model.add(Conv2D(96, (7, 7),strides=2, padding='same',
                 activation='relu',input_shape=(200, 200,num_channels),
                name='input_layer'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(48, (5, 5),strides=1, padding='same',activation='relu'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(24, (3, 3),strides=1, padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(24, (3, 3),strides=1, padding='same',activation='relu'))
model.add(Flatten())
# # Spatial Pooling layer to deal with differing image sizes
# model.add(SpatialPyramidPooling([6, 3, 2,1]))

model.add(Dense(128,activation='relu'))

# Classification layer
model.add(Dense(2, activation='softmax'))

callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
# opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


hist = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),verbose=1,
                    callbacks=callbacks)




Epoch 1/10
46/46 [==============================] - 17s 360ms/step - loss: 0.0498 - acc: 0.9783 - val_loss: 0.3321 - val_acc: 0.9189
Epoch 2/10
46/46 [==============================] - 19s 418ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0599 - val_acc: 1.0000
Epoch 3/10
46/46 [==============================] - 16s 353ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 4/10
46/46 [==============================] - 15s 325ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 5/10
46/46 [==============================] - 15s 322ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 6/10
46/46 [==============================] - 15s 329ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 7/10
46/46 [==============================] - 15s 324ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 8/10
46/46 [===========================

### Utilize transfer learning

In [48]:
filepath = "caltech_101_raw_3_channel_no_pyramid.h5"
trans_model = load_model(filepath)

# Allow no weight adjustments for the pretrained layers
for layer in trans_model.layers:
    layer.trainable = False

    
# Spatial Pooling layer to deal with differing image sizes
# trans_model.add(SpatialPyramidPooling([1, 2, 4]))
trans_model.add(Conv2D(32,(3,3)))
trans_model.add(MaxPooling2D((1,1)))
trans_model.add(Flatten())
trans_model.add(Dense(128,activation='relu'))
# Classification Layer
trans_model.add(Dense(num_classes, activation='softmax'))

C:\Users\gjber\AppData\Roaming\Python\Python36\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [49]:
callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 10
# Fit model
trans_hist = trans_model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),verbose=1,
                    callbacks=callbacks)

Epoch 1/10
46/46 [==============================] - 9s 201ms/step - loss: 1.1073 - acc: 0.6848 - val_loss: 0.7385 - val_acc: 0.1622
Epoch 2/10
46/46 [==============================] - 8s 163ms/step - loss: 0.7023 - acc: 0.7880 - val_loss: 0.7603 - val_acc: 0.4865
Epoch 3/10
46/46 [==============================] - 8s 165ms/step - loss: 0.5840 - acc: 0.7826 - val_loss: 0.7906 - val_acc: 0.4865
Epoch 4/10
46/46 [==============================] - 8s 165ms/step - loss: 0.5051 - acc: 0.8152 - val_loss: 0.7999 - val_acc: 0.4865
Epoch 5/10
46/46 [==============================] - 9s 185ms/step - loss: 0.5345 - acc: 0.8098 - val_loss: 0.7399 - val_acc: 0.4865
Epoch 6/10
38/46 [=======================>......] - ETA: 0s - loss: 0.4493 - acc: 0.7763

KeyboardInterrupt: 

Evaluate

In [ ]:
trans_pred = []
reg_pred= []
for img in x_test:
    trans_pred.append(trans_model.predict(np.expand_dims(img, axis=0)))
    reg_pred.append(model.predict(np.expand_dims(img, axis=0)))

reg_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in reg_pred]
trans_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in trans_pred]

y_test = ["Hunter" if x[0] > x[1] else "Alex" for x in y_test]

In [ ]:
from sklearn.metrics import classification_report
print("Model with No transfer learning")
print(classification_report(y_test, reg_pred))
print("Model with transfer learning")
print(classification_report(y_test, trans_pred))